In [4]:
import os

In [5]:
taco_path = "/home/tensorflow/waste/datasets/taco_dataset/"

In [3]:
import json

In [6]:
taco_ann_raw = json.load(open(os.path.join(taco_path,'annotations.json'), 'r'))

In [13]:
image_annotations = {}
for img_ann in taco_ann_raw['annotations'][:min(1000, len(taco_ann_raw['annotations']))]:
    img_id = img_ann['image_id']
    img_bbox = img_ann['bbox']
    if img_id in image_annotations:
        image_annotations[img_id]['bboxes'].append(img_bbox)
    else:
        img_desc = taco_ann_raw['images'][img_id]
        img_link = img_desc['flickr_url']
        img_height = img_desc['height']
        img_width = img_desc['width']
        img_name = img_link.split('/')[-1]
        img_path = os.path.join(taco_path, 'train', img_name)
        if os.path.exists(img_path):
            img_category = img_ann['category_id']
            image_annotations[img_id] = {"name":img_name, "bboxes":[img_bbox],
                                            "category_id":img_category,
                                            "width":img_width,
                                          "height":img_height}

In [15]:
len(image_annotations)

290

Translate the annotations into format for training the cascade filter

In [34]:
with open(os.path.join(taco_path, 'cascade_training_info.dat'), 'w') as f:
    for img_an in image_annotations.values():
        f.write(f"{os.path.join(taco_path, 'train', img_an['name'])}\t")
        f.write(f"{len(img_an['bboxes'])}\t")
        for bbox in img_an['bboxes']:
            xs, ys, xe, ye = bbox
            w = xe-xs
            h = ye-ys
            f.write(f"{int(xs if w>0 else xe)} {int(ys if h>0 else ye)} {int(abs(w))} {int(abs(h))}\t")
        f.write('\n')

### Background images

In [25]:
dataset = '/home/tensorflow/waste/datasets/VOCdevkit/VOC2007/'

In [27]:
categories = ['bird', 'boat', 'bus', 'car', 'cow', 'horse', 'sheep', 'train']

In [28]:
img_names = []
for cat in categories:
    cat_file_list = os.path.join(dataset, 'ImageSets/Main', f'{cat}_train.txt')
    with open(cat_file_list, 'r') as f:
        for l in f:
            fields = l.strip().split()
            present = int(fields[1])
            if present > 0: img_names.append(fields[0])

In [31]:
with open(os.path.join(taco_path, 'bg_cascade.txt'), 'w') as f:
    for img_name in img_names:
        f.write(f"{os.path.join(dataset,'JPEGImages',img_name)}\n")